In [1]:
from korbash_lib import PowerMeter, TensionGauge, Motor, MotorSystem, Puller, start, end, Everage, Exp_everage
import thorlabs_apt as apt
import time
import numpy as np
import pandas as pd
import datetime
import PySimpleGUI as sg
from tkinter import messagebox as mb
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
from scipy import stats
output_notebook()

Loading BokehJS ...

In [45]:
ZERO_WEIGHT = 0 # 2438.309 ##1068.4573
WEIGHT_COEF = -0.0075585384235655265  ## ТЕНЗО ЗАЖИМ
tg = TensionGauge(ZERO_WEIGHT, WEIGHT_COEF)
pm = PowerMeter()
#print(apt.list_available_devices())
R = 90113195
L = 90113196
M = 90113197
motorR = Motor(R, "right motor", 70)
motorL = Motor(L, "left motor")
motorM = Motor(M, "motor",75) # начальная позиция 15
motors=MotorSystem(motorR,motorL,motorM)
#start()
dt = 0.1
t = 0
tPoints = np.array([])
dL_points = np.array([])
a_points = np.array([])
v_points = np.array([])
data = pd.DataFrame({
     'time': tPoints,
     'power': pm.powers,
     'tension': tg.weights,
     'motorR_coord': motorR.points,
     'motorL_coord': motorL.points,
     'motorM_coord': motorR.points,
     'dL_coord': dL_points,
     'a': a_points,
     'v': v_points,
})
pl_motor_t = figure(plot_width=980, plot_height=400, sizing_mode="scale_both")
pl_m_r = pl_motor_t.line(x='time',y='motorR_coord' ,source=data, legend_label="motorR", line_color="red")
pl_m_l = pl_motor_t.line(x='time',y='motorL_coord' ,source=data, legend_label="motorL", line_color="blue")
pl_m_m = pl_motor_t.line(x='time',y='dL_coord' ,source=data, legend_label="dL_coord", line_color="green")
pl_power_t = figure(plot_width=980, plot_height=200, sizing_mode="scale_both")
p = pl_power_t.line(x='time', y='power',source=data)
pl_tension_t = figure(plot_width=980, plot_height=200, sizing_mode="scale_both")
p2 = pl_tension_t.line(x='time', y='tension',source=data)
#grid = gridplot([pl_motor_t, pl_power_t], ncols=2)
grid=layout([
    [pl_motor_t],
    [pl_power_t],
    [pl_tension_t],
])

NameError: b'Insufficient location information or the device or resource is not present in the system.'

In [15]:
tPoints = np.array([])
t0 = time.time()
i=0
tau=10
while time.time()-t0 < tau:
    tg.ReadTension()
    tPoints = np.append(tPoints, time.time() - t0)
    i+=1
    #if(i%10==0):
    #print(tg.weights[-1],Everage(tg.weights,tPoints,tau))
tg.zeroWeight+=Everage(tg.weights,tPoints,tau)
print(tg.zeroWeight)
tPoints = np.array([])
tg.Clear()

6.448817607616377


0

In [42]:
tg.ReadTension()

2.238437358740189

In [28]:
tg.Clear()
tPoints = np.array([])

In [4]:
motorM.MoveTo(20)

0

In [46]:
del tg
del pm

NameError: name 'pm' is not defined

In [ ]:
def stop_btn2_click(stop_btn2):
    motors.Stop()
stop_btn2 = ipywidgets.Button(description='stop')
stop_btn2.on_click(stop_btn2_click)
display(stop_btn2)

In [9]:
motors.MoveToStart(60)
#print(tg.SetZeroWeight())

In [ ]:
motorR.Getposition()
motorR.MoveTo(motorR.points[-1]+0.1)

In [ ]:
w_points=np.array([])
x_points=np.array([])

In [27]:
t0 = time.time()
i=0
tau=0.3
wide=100
k=277
dw=1
while abs(dw) > 0.1:
    t1=time.time()
    while time.time()-t1 < tau:
        tg.ReadTension()
        tPoints = np.append(tPoints, time.time() - t0)
    w=Everage(tg.weights,tPoints,tau)
    print(w)
    dw=wide-w
    dx=dw/k
    #print(dx)
    motorR.Getposition()
    motorR.MoveTo(motorR.points[-1]-dx,a=1)
    while motorR.is_in_motion:
        pass
    time.sleep(0.1)
while time.time()-t1 < tau:
    tg.ReadTension()
    tPoints = np.append(tPoints, time.time() - t0)
w=Everage(tg.weights,tPoints,tau)


'''motorR.Getposition()
motorL.Getposition()
w_points = np.append(w_points, w)
x_points = np.append(x_points,200 -motorR.points[-1] -motorL.points[-1])
k_data=pd.DataFrame({
         'w': w_points,
         'x': x_points,
        })
k_data.plot(x='x',y='w')'''

37.60258381861319
105.7453624930436
101.7676816476422
99.91977017384893


"motorR.Getposition()\nmotorL.Getposition()\nw_points = np.append(w_points, w)\nx_points = np.append(x_points,200 -motorR.points[-1] -motorL.points[-1])\nk_data=pd.DataFrame({\n         'w': w_points,\n         'x': x_points,\n        })\nk_data.plot(x='x',y='w')"

In [10]:
motors.Move(-30)

0

In [11]:
stop_fl = False
fl1=False
def stop_btn_click(stop_btn):
    global stop_fl
    stop_fl = True
    stop_btn.description = 'done'
    
def btn2_click(btn2):
    global fl1
    fl1 = True    

stop_btn = ipywidgets.Button(description='stop')
stop_btn.on_click(stop_btn_click)
btn2 = ipywidgets.Button(description='btn2')
btn2.on_click(btn2_click)

show(grid, notebook_handle=True)
sl_v=ipywidgets.FloatSlider(max=10,value=2)
sl_a=ipywidgets.FloatSlider(max=18,value=8)
sl_dt=ipywidgets.FloatSlider(min=-100, max=100,value=0)
output2 = ipywidgets.Output()
display(sl_v,sl_a,sl_dt, stop_btn, btn2, output2)
t0 = time.time()
x_coord = [20, 10]
num = 0
v=sl_v.value
a=sl_a.value
print(a,v)
while motors.IsInMotion():
    pass
with ui_events() as poll:
    while time.time() - t0 < 1351:
        tPoints = np.append(tPoints, time.time() - t0)
        tg.ReadTension()
        pm.ReadPower()
        motorR.Getposition()
        motorL.Getposition()
        motorM.Getposition()
        a_points = np.append(a_points,a)
        v_points = np.append(v_points,v)
        dL_points = np.append(dL_points,-motorR.points[-1]-motorL.points[-1]-(-motorR.points[0]-motorL.points[0]))
        pl_m_m.data_source.data=pl_m_l.data_source.data=pl_m_r.data_source.data=p2.data_source.data = p.data_source.data = data = pd.DataFrame({
         'time': tPoints,
         'power': pm.powers,
         'tension': tg.weights,
         'motorR_coord': motorR.points,
         'motorL_coord': motorL.points,
         'motorM_coord': motorM.points,
         'dL_coord': dL_points,
         'a': a_points,
         'v': v_points,
        })
        push_notebook()
        poll(10)
        v=sl_v.value
        a=sl_a.value
        dt=sl_dt.value/10000
        # with output2:
        #    print(v,end=" ")
        if stop_fl == True:
            motors.Stop()
            break
        if fl1 == True:
            motorR.set_velocity_parameters(0, a_norm, v_norm)
            motorR.move_to(60)
            fl1=False
        if not motors.IsInMotion():
            if(num%24<12):
                motors.Move(5,v,a,0,a**2*dt/v)
            else:
                motors.Move(-5,v,a,0,a**2*dt/v)
            num+=1
        #update(data)
print("the end")     

FloatSlider(value=2.0, max=10.0)

FloatSlider(value=8.0, max=18.0)

FloatSlider(value=0.0, min=-100.0)

Button(description='stop', style=ButtonStyle())

Button(description='btn2', style=ButtonStyle())

Output()

8.0 2.0
the end


In [7]:
q = datetime.datetime.today()
st = str(q.day) + '_' + str(q.month) + '_' + str(q.year) + '__' + str(q.hour) + '_' + str(q.minute)
st = r'C:\Users\Fiber\Desktop\table_control_data\test\data_' + st + '.csv'
data.to_csv(st, index=False)
del tg
del pm